# Dataset Simulation Test 

### Based off of Literature review and research into distributions for pet dogs in Southern US

#### Note:
- Dog-specific features use national distributions
- Environ features use Southeast US distributions

In [104]:
import numpy as np
import pandas as pd
import ast 

In [105]:
import os
os.getcwd()
# change directory to where you cloned Repo on your local machine
# you can then access our files there rather than needing to download each onto local machine
os.chdir("/Users/ellawileman/Documents/fall2025/VIPR/CoolDawgs_PersonalizationGrp")

In [106]:
dogs_dataset = pd.read_csv("dogs_dataset.csv")
dogs_dataset.head()

# use for sampling breed.
# for each breed. this dataset is simulated but menat to represent real-world
# distributions

,Breed,Age (Years),Weight (kg),Color,Gender
0,Airedale Terrier,13,35,White,Male
1,Jack Russell Terrier,10,43,Tan,Female
2,Dogo Argentino,2,16,Spotted,Female
3,Labrador Retriever,9,57,Bicolor,Male
4,French Bulldog,12,39,Spotted,Male


In [107]:
# get distribution for breeds (popularity in US = proportions)
breed_probs = dogs_dataset["Breed"].value_counts(normalize=True)

# View as percentages
breed_percentages = (breed_probs * 100).round(2)
# Print
print(breed_percentages.head())  # top 5 breeds

breed_categories = breed_probs.index.tolist()
breed_probabilities = breed_probs.values.tolist()

Breed
Rottweiler              3.93
French Bulldog          2.33
Pekingese               2.27
Pembroke Welsh Corgi    2.23
Doberman Pinscher       2.20
Name: proportion, dtype: float64


In [108]:
# summarize each breed's age, wt, gender, color distributions.
breed_params_df = dogs_dataset.groupby('Breed').agg({
    'Age (Years)': ['mean', 'std'],
    'Weight (kg)': ['mean', 'std'],
    'Gender': lambda x: x.value_counts(normalize=True).to_dict(),
    'Color': lambda x: x.value_counts(normalize=True).to_dict()
}).reset_index()

# need mu and sd to use normal distribution for each breed's age and wt to build sample population.
breed_params_df.columns = ['Breed', 'age_mu', 'age_sd', 'wt_mu', 'wt_sd', 'gender_probs', 'color_probs']
breed_params_df.head()

,Breed,age_mu,age_sd,wt_mu,wt_sd,gender_probs,color_probs
0,Airedale Terrier,8.218182,4.336285,31.818182,13.997475,"{'Male': 0.5454545454545454, 'Female': 0.45454...","{'Tricolor': 0.12727272727272726, 'Bicolor': 0..."
1,Akita,7.784314,3.874603,31.686275,14.360348,"{'Male': 0.5686274509803921, 'Female': 0.43137...","{'Merle': 0.11764705882352941, 'Black and Whit..."
2,Alaskan Malamute,6.981818,3.870548,34.636364,14.876484,"{'Male': 0.5454545454545454, 'Female': 0.45454...","{'Blue': 0.10909090909090909, 'Merle': 0.10909..."
3,Australian Shepherd,7.823529,3.675355,33.352941,16.359491,"{'Male': 0.5882352941176471, 'Female': 0.41176...","{'Brown': 0.11764705882352941, 'Sable': 0.0980..."
4,Basenji,7.551020,4.178622,31.938776,17.336628,"{'Male': 0.5102040816326531, 'Female': 0.48979...","{'Cream': 0.12244897959183673, 'Brindle': 0.12..."


#### ***TODO***: get breed specific max and mins. so we can ensure sampled values are within reasonable range. 
- trim off the S at end of each breed (plural) from breed traits dataset and convert Retrievers (Golden) to Golden Retriever, etc to match with our sampled breed names in synthetic dataset

In [109]:
breed_schema = {
    "Breed": {
        "type": "categorical",
        "categories": breed_categories,
        "probs": breed_probabilities
    }
}
len(breed_schema["Breed"]["categories"])
# only 53 breeds here
sum(breed_schema["Breed"]["probs"])
display(breed_schema["Breed"]["categories"])

['Rottweiler',
 'French Bulldog',
 'Pekingese',
 'Pembroke Welsh Corgi',
 'Doberman Pinscher',
 'Weimaraner',
 'Pug',
 'Bichon Frise',
 'Vizsla',
 'Cavalier King Charles Spaniel',
 'Chinese Shar-Pei',
 'Poodle',
 'Whippet',
 'Dachshund',
 'Cocker Spaniel',
 'Bloodhound',
 'Border Collie',
 'Lhasa Apso',
 'Samoyed',
 'Bernese Mountain Dog',
 'Alaskan Malamute',
 'Miniature Schnauzer',
 'Jack Russell Terrier',
 'Bulldog',
 'Airedale Terrier',
 'Dogo Argentino',
 'Irish Setter',
 'Bull Terrier',
 'Havanese',
 'Beagle',
 'Great Dane',
 'Saint Bernard',
 'Boston Terrier',
 'German Shepherd',
 'Siberian Husky',
 'Yorkshire Terrier',
 'Labrador Retriever',
 'Boxer',
 'Chesapeake Bay Retriever',
 'Maltese',
 'Shih Tzu',
 'Australian Shepherd',
 'Akita',
 'Shetland Sheepdog',
 'Basenji',
 'Papillon',
 'West Highland White Terrier',
 'Chihuahua',
 'Pomeranian',
 'Schnauzer',
 'Shiba Inu',
 'Belgian Malinois',
 'Golden Retriever']

### Using sample dog pop (aggregate by breed to find params) from Kaggle

In [110]:
N = 1000 # can change sample size later
sampled_breeds = np.random.choice(
        breed_schema["Breed"]["categories"],
        size=N,
        p=breed_schema["Breed"]["probs"]
    )
#display(breeds)

synthetic_rows = []

for breed in sampled_breeds:
    # lookup parameters for that breed
    params = breed_params_df.loc[breed_params_df["Breed"] == breed].iloc[0]

    # sample continuous features
    age = np.random.normal(params["age_mu"], params["age_sd"])
    weight = np.random.normal(params["wt_mu"], params["wt_sd"])

    # handle categorical features
    gender_probs = params["gender_probs"]
    if isinstance(gender_probs, str):
        gender_probs = ast.literal_eval(gender_probs)
    gender = np.random.choice(list(gender_probs.keys()), p=list(gender_probs.values()))

    color_probs = params["color_probs"]
    if isinstance(color_probs, str):
        color_probs = ast.literal_eval(color_probs)
    color = np.random.choice(list(color_probs.keys()), p=list(color_probs.values()))

    synthetic_rows.append({
        "Breed": breed,
        "Age": round(age, 1),
        "Weight": round(weight, 1),
        "Gender": gender,
        "Color": color
    })
synthetic_dogs_df = pd.DataFrame(synthetic_rows)
display(synthetic_dogs_df.head(30))

,Breed,Age,Weight,Gender,Color
0,Bulldog,10.5,12.7,Female,Black and Tan
1,Akita,1.9,39.9,Female,Black and White
2,Pomeranian,12.4,45.4,Female,Tan
3,Siberian Husky,6.3,24.5,Male,Black
4,Beagle,7.0,53.6,Male,Red
5,Irish Setter,14.4,40.2,Male,Spotted
6,Alaskan Malamute,4.8,6.4,Male,Tan
7,Shetland Sheepdog,3.3,27.1,Female,Gray
8,Miniature Schnauzer,9.2,23.3,Male,Tricolor
9,Chesapeake Bay Retriever,8.5,42.0,Female,Cream


### Now map to coat length, coat type, and playfulness/energy level*

*From other Kaggle dataset(s) - breed_traits.csv

In [111]:
breed_traits = pd.read_csv("breed_traits.csv")
breed_traits.columns
#display(breed_traits)
trait_desc = pd.read_csv("trait_description.csv")
display(trait_desc)

# explains scale of each trait (scale of 1-5 and what 1 and 5 represent)
# for each dog, have their score in each of these columns. (195 breeds)
# Let's use for Coat Length, Coat Type, Energy Level, Playfulness Level

,Trait,Trait_1,Trait_5,Description
0,Affectionate With Family,Independent,Lovey-Dovey,How affectionate a breed is likely to be with ...
1,Good With Young Children,Not Recommended,Good With Children,A breed's level of tolerance and patience with...
2,Good With Other Dogs,Not Recommended,Good With Other Dogs,How generally friendly a breed is towards othe...
3,Shedding Level,No Shedding,Hair Everywhere,How much fur and hair you can expect the breed...
4,Coat Grooming Frequency,Monthly,Daily,"How frequently a breed requires bathing, brush..."
5,Drooling Level,Less Likely to Drool,Always Have a Towel,How drool-prone a breed tends to be. If you're...
6,Coat Type,-,-,"Canine coats come in many different types, dep..."
7,Coat Length,-,-,How long the breed's coat is expected to be. S...
8,Openness To Strangers,Reserved,Everyone Is My Best Friend,How welcoming a breed is likely to be towards ...
9,Playfulness Level,Only When You Want To Play,Non-Stop,How enthusiastic about play a breed is likely ...


In [112]:
breed_traits["Coat Length"].value_counts()


Coat Length
Short           86
Medium          79
Long            29
Plott Hounds     1
Name: count, dtype: int64

In [113]:
breed_traits["Coat Type"].value_counts()

Coat Type
Double          66
Smooth          66
Wiry            30
Silky            9
Curly            7
Wavy             6
Corded           4
Hairless         3
Rough            3
Plott Hounds     1
Name: count, dtype: int64

In [114]:
sorted_breeds = sorted(breed_traits["Breed"].unique())
pd.set_option("display.max_rows", None)  # show all
display(sorted_breeds)


['Affenpinschers',
 'Afghan\xa0Hounds',
 'Airedale\xa0Terriers',
 'Akitas',
 'Alaskan\xa0Malamutes',
 'American\xa0English\xa0Coonhounds',
 'American\xa0Eskimo\xa0Dogs',
 'American\xa0Foxhounds',
 'American\xa0Hairless\xa0Terriers',
 'American\xa0Staffordshire\xa0Terriers',
 'Anatolian\xa0Shepherd\xa0Dogs',
 'Australian\xa0Cattle\xa0Dogs',
 'Australian\xa0Shepherds',
 'Australian\xa0Terriers',
 'Azawakhs',
 'Barbets',
 'Basenjis',
 'Basset\xa0Hounds',
 'Beagles',
 'Bearded\xa0Collies',
 'Beaucerons',
 'Bedlington\xa0Terriers',
 'Belgian\xa0Malinois',
 'Belgian\xa0Sheepdogs',
 'Belgian\xa0Tervuren',
 'Bergamasco\xa0Sheepdogs',
 'Berger\xa0Picards',
 'Bernese\xa0Mountain\xa0Dogs',
 'Bichons\xa0Frises',
 'Black\xa0Russian\xa0Terriers',
 'Black\xa0and\xa0Tan\xa0Coonhounds',
 'Bloodhounds',
 'Bluetick\xa0Coonhounds',
 'Boerboels',
 'Border\xa0Collies',
 'Border\xa0Terriers',
 'Borzois',
 'Boston\xa0Terriers',
 'Bouviers\xa0des\xa0Flandres',
 'Boxers',
 'Briards',
 'Brittanys',
 'Brussels\xa

In [115]:
# Clean breed names in both datasets
def clean_breed_names(series):
    return (series
            .str.replace('\xa0', ' ', regex=False)   # replace non-breaking space
            .str.strip()                              # remove leading/trailing spaces
           )

breed_traits["Breed"] = clean_breed_names(breed_traits["Breed"])
dogs_dataset["Breed"] = clean_breed_names(dogs_dataset["Breed"])

breed_traits["Breed"].value_counts()

Breed
Retrievers (Labrador)                    1
Swedish Vallhunds                        1
Bearded Collies                          1
Black Russian Terriers                   1
Black and Tan Coonhounds                 1
Spaniels (Welsh Springer)                1
American Hairless Terriers               1
Norfolk Terriers                         1
Xoloitzcuintli                           1
Manchester Terriers                      1
Kerry Blue Terriers                      1
Australian Terriers                      1
Spaniels (Clumber)                       1
Lakeland Terriers                        1
Bluetick Coonhounds                      1
English Toy Spaniels                     1
German Pinschers                         1
Tibetan Mastiffs                         1
Bedlington Terriers                      1
Greyhounds                               1
Pulik                                    1
Salukis                                  1
Barbets                                  1
Fox T

In [116]:
# find all rows that have parentheses
breed_traits_with_parens = breed_traits[breed_traits['Breed'].str.contains(r'\(|\)', na=False)]
print(breed_traits_with_parens)

#.str.replace(r'[^a-z\s]', '', regex=True) # remove punctuation if needed

                                     Breed  Affectionate With Family  \
0                    Retrievers (Labrador)                         5   
3                      Retrievers (Golden)                         5   
8            Pointers (German Shorthaired)                         5   
25             Spaniels (English Springer)                         5   
29                       Spaniels (Cocker)                         4   
46               Spaniels (English Cocker)                         5   
49             Retrievers (Chesapeake Bay)                         4   
59            Pointers (German Wirehaired)                         5   
74                         Setters (Irish)                         5   
86                       Spaniels (Boykin)                         3   
89   Retrievers (Nova Scotia Duck Tolling)                         5   
100                      Setters (English)                         5   
101               Retrievers (Flat-Coated)                      

In [117]:
# fixing paretheses 
conditions = [
    (breed_traits['Breed'] == "Retrievers (Golden)"),
    (breed_traits['Breed'] == "Retrievers (Labrador)"),
    (breed_traits['Breed'] == "Retrievers (Chesapeake Bay)"),
    (breed_traits['Breed'] == 'Spaniels (Cocker)')
]
# TODO: add more cases!!

choices = ['Golden Retrievers', 'Labrador Retrievers', 'Chesapeake Bay Retrievers', 'Cocker Spaniels']

breed_traits['Breed_clean'] = np.select(conditions, choices, default=breed_traits['Breed'])

# Filter rows where 'Breed_clean' does not contain '(' or ')'
no_parentheses = breed_traits[~breed_traits["Breed_clean"].str.contains(r'\(|\)', na=False)]
# remove duplicates
no_parentheses = no_parentheses.drop_duplicates(subset=["Breed_clean"])
no_parentheses["Breed_singular"] =  no_parentheses['Breed_clean'].str[:-1]
#breed_traits["Breed_singular"]

In [118]:
print(len(no_parentheses))
no_parentheses["Breed_singular"].value_counts()

175


Breed_singular
Labrador Retriever                 1
Australian Terrier                 1
Bearded Collie                     1
Black Russian Terrier              1
Black and Tan Coonhound            1
American Hairless Terrier          1
Norfolk Terrier                    1
Xoloitzcuintl                      1
Manchester Terrier                 1
Kerry Blue Terrier                 1
Lakeland Terrier                   1
Afghan Hound                       1
Bluetick Coonhound                 1
English Toy Spaniel                1
German Pinscher                    1
Tibetan Mastiff                    1
Bedlington Terrier                 1
Greyhound                          1
Puli                               1
Saluki                             1
Boerboel                           1
Beauceron                          1
Redbone Coonhound                  1
Spinoni Italian                    1
Leonberger                         1
Tibetan Terrier                    1
Neapolitan Mastiff     

In [119]:
no_parentheses["Breed_singular"].value_counts()

Breed_singular
Labrador Retriever                 1
Australian Terrier                 1
Bearded Collie                     1
Black Russian Terrier              1
Black and Tan Coonhound            1
American Hairless Terrier          1
Norfolk Terrier                    1
Xoloitzcuintl                      1
Manchester Terrier                 1
Kerry Blue Terrier                 1
Lakeland Terrier                   1
Afghan Hound                       1
Bluetick Coonhound                 1
English Toy Spaniel                1
German Pinscher                    1
Tibetan Mastiff                    1
Bedlington Terrier                 1
Greyhound                          1
Puli                               1
Saluki                             1
Boerboel                           1
Beauceron                          1
Redbone Coonhound                  1
Spinoni Italian                    1
Leonberger                         1
Tibetan Terrier                    1
Neapolitan Mastiff     

In [120]:
no_parentheses.drop(columns=["Breed", "Breed_clean"], inplace=True)
no_parentheses.rename(columns={"Breed_singular": "Breed"}, inplace=True)
no_parentheses.head(5)

,Affectionate With Family,Good With Young Children,Good With Other Dogs,Shedding Level,Coat Grooming Frequency,Drooling Level,Coat Type,Coat Length,Openness To Strangers,Playfulness Level,Watchdog/Protective Nature,Adaptability Level,Trainability Level,Energy Level,Barking Level,Mental Stimulation Needs,Breed
0,5,5,5,4,2,2,Double,Short,5,5,3,5,5,5,3,4,Labrador Retriever
1,5,5,4,3,1,3,Smooth,Short,5,5,3,5,4,3,1,3,French Bulldog
2,5,5,3,4,2,2,Double,Medium,3,4,5,5,5,5,3,5,German Shepherd Dog
3,5,5,5,4,2,2,Double,Medium,5,4,3,5,5,3,1,4,Golden Retriever
4,4,3,3,3,3,3,Smooth,Short,4,4,3,3,4,3,2,3,Bulldog


In [121]:

breed_traits = no_parentheses 

breed_traits_breednames = set(breed_traits["Breed"])
dogs_breeds = set(dogs_dataset["Breed"])

matches = breed_traits_breednames & dogs_breeds
print(f"Exact matches after cleaning: {len(matches)}")

# Find close matches for non-matching breeds
from difflib import get_close_matches
non_matching_breeds = breed_traits_breednames - matches   
close_matches = {breed: get_close_matches(breed, dogs_breeds, n=1, cutoff=0.8) for breed in non_matching_breeds}
# Filter out breeds with no close matches
close_matches = {k: v[0] for k, v in close_matches.items() if v}
print("Close matches found:")
for k, v in close_matches.items():
    print(f"{k} -> {v}")


Exact matches after cleaning: 39
Close matches found:
Russell Terrier -> Jack Russell Terrier
Havanes -> Havanese
German Pinscher -> Doberman Pinscher
St. Bernard -> Saint Bernard
Icelandic Sheepdog -> Shetland Sheepdog
German Shepherd Dog -> German Shepherd
Siberian Huskie -> Siberian Husky
Bichons Frise -> Bichon Frise
Maltes -> Maltese
Pekinges -> Pekingese
Bearded Collie -> Border Collie
Parson Russell Terrier -> Jack Russell Terrier
Belgian Malinoi -> Belgian Malinois
Shiba In -> Shiba Inu
Shih Tz -> Shih Tzu
Chinese Shar-Pe -> Chinese Shar-Pei


In [122]:
# Manually verify and create a mapping dictionary
mapping_dict = {
    "Bearded Collie": "Border Collie",
    "Siberian Huskie": "Siberian Husky",
    "Pekinges": "Pekingese",
    "Havanese": "Havanese",
    "German Shepard Dog": "German Shepherd",
    "Maltes": "Maltese",
    "Shiba Inu": "Shiba Inu",
    "St. Bernard": "Saint Bernard",
    "German Pinscher": "Doberman Pinscher",
    "Russell Terrier": "Jack Russell Terrier",
    "Icelandic Sheepdog": "Shetland Sheepdog",
    "Chinese Shar-Pe": "Chinese Shar-Pei",
    "Havanes": "Havanese",
    "Kuvaszo": "Kuvasz",
    "Lowche": "Lowchen",
    "Belgian Malinois": "Belgian Malinois"
    # Add more mappings as needed - can do later on.
}

# Apply the mapping to the breed_traits DataFrame
breed_traits["Breed"] = breed_traits["Breed"].replace(mapping_dict)

# Recalculate matches after applying the mapping
all_breeds_from_traits = set(breed_traits["Breed"])
matches = all_breeds_from_traits & dogs_breeds
print(f"Exact matches after mapping: {len(matches)}")

Exact matches after mapping: 46


#### ***TODO*** Examining breed names so we can match the between data sources
- Need to fix mapping of dog breeds (standardize names so we can join on breeds between datasets)
- Breed is plural in one dataset
- some breeds need to be generalized (should hit 53)

#### Wait until breed names standardized to join them!!

In [123]:

check = pd.merge(synthetic_dogs_df, breed_traits, on="Breed", how="left")
notnull = check[check["Coat Length"].notna()]
len(notnull)


935

In [124]:
# not null:
print("Num breeds represented in synthetic dataset that have trait data:")
print(len(notnull["Breed"].unique()))
notnull["Breed"].value_counts()
# only 3 breeds actually matched exactly

Num breeds represented in synthetic dataset that have trait data:
46


Breed
Doberman Pinscher                52
Rottweiler                       45
Shetland Sheepdog                44
Border Collie                    34
Weimaraner                       30
Basenji                          28
Airedale Terrier                 26
French Bulldog                   25
Dachshund                        25
Bull Terrier                     25
Cocker Spaniel                   25
Poodle                           23
Pekingese                        22
Cavalier King Charles Spaniel    22
Saint Bernard                    22
Lhasa Apso                       22
Bernese Mountain Dog             21
Chinese Shar-Pei                 21
Pomeranian                       19
Pug                              19
Australian Shepherd              19
Golden Retriever                 18
Samoyed                          18
Pembroke Welsh Corgi             18
Miniature Schnauzer              17
Labrador Retriever               17
Vizsla                           17
Bloodhound            

### Sample for time of year (season) so we can get different environ distributions

In [125]:
notnull.head(10)

,Breed,Age,Weight,Gender,Color,Affectionate With Family,Good With Young Children,Good With Other Dogs,Shedding Level,Coat Grooming Frequency,...,Coat Type,Coat Length,Openness To Strangers,Playfulness Level,Watchdog/Protective Nature,Adaptability Level,Trainability Level,Energy Level,Barking Level,Mental Stimulation Needs
0,Bulldog,10.5,12.7,Female,Black and Tan,4.0,3.0,3.0,3.0,3.0,...,Smooth,Short,4.0,4.0,3.0,3.0,4.0,3.0,2.0,3.0
1,Akita,1.9,39.9,Female,Black and White,3.0,3.0,1.0,3.0,3.0,...,Double,Medium,2.0,3.0,5.0,3.0,3.0,4.0,2.0,3.0
2,Pomeranian,12.4,45.4,Female,Tan,5.0,3.0,3.0,2.0,3.0,...,Double,Long,3.0,3.0,4.0,4.0,3.0,3.0,4.0,3.0
3,Siberian Husky,6.3,24.5,Male,Black,5.0,5.0,5.0,4.0,2.0,...,Double,Medium,5.0,5.0,1.0,4.0,3.0,5.0,5.0,4.0
4,Beagle,7.0,53.6,Male,Red,3.0,5.0,5.0,3.0,2.0,...,Smooth,Short,3.0,4.0,2.0,4.0,3.0,4.0,4.0,4.0
6,Alaskan Malamute,4.8,6.4,Male,Tan,3.0,3.0,3.0,3.0,3.0,...,Double,Medium,3.0,3.0,4.0,3.0,5.0,4.0,3.0,3.0
7,Shetland Sheepdog,3.3,27.1,Female,Gray,5.0,5.0,5.0,3.0,3.0,...,Double,Long,2.0,5.0,5.0,5.0,5.0,4.0,5.0,4.0
8,Shetland Sheepdog,3.3,27.1,Female,Gray,5.0,5.0,4.0,3.0,2.0,...,Double,Medium,3.0,3.0,4.0,4.0,5.0,4.0,4.0,5.0
9,Miniature Schnauzer,9.2,23.3,Male,Tricolor,5.0,5.0,3.0,3.0,4.0,...,Wiry,Medium,3.0,4.0,4.0,5.0,5.0,3.0,5.0,4.0
10,Chesapeake Bay Retriever,8.5,42.0,Female,Cream,4.0,3.0,3.0,3.0,3.0,...,Wiry,Medium,3.0,3.0,4.0,4.0,5.0,4.0,3.0,4.0


In [126]:
from_kaggle_sets = notnull[["Breed", "Age", "Weight", "Gender", "Color", "Coat Length", "Coat Type", "Energy Level", "Playfulness Level"]]
from_kaggle_sets.head(10)

,Breed,Age,Weight,Gender,Color,Coat Length,Coat Type,Energy Level,Playfulness Level
0,Bulldog,10.5,12.7,Female,Black and Tan,Short,Smooth,3.0,4.0
1,Akita,1.9,39.9,Female,Black and White,Medium,Double,4.0,3.0
2,Pomeranian,12.4,45.4,Female,Tan,Long,Double,3.0,3.0
3,Siberian Husky,6.3,24.5,Male,Black,Medium,Double,5.0,5.0
4,Beagle,7.0,53.6,Male,Red,Short,Smooth,4.0,4.0
6,Alaskan Malamute,4.8,6.4,Male,Tan,Medium,Double,4.0,3.0
7,Shetland Sheepdog,3.3,27.1,Female,Gray,Long,Double,4.0,5.0
8,Shetland Sheepdog,3.3,27.1,Female,Gray,Medium,Double,4.0,3.0
9,Miniature Schnauzer,9.2,23.3,Male,Tricolor,Medium,Wiry,3.0,4.0
10,Chesapeake Bay Retriever,8.5,42.0,Female,Cream,Medium,Wiry,4.0,3.0


In [127]:
np.shape(from_kaggle_sets)

(935, 9)

In [128]:
# weight in kilograms 

conditions = [
    (from_kaggle_sets['Weight'] < 4.5),
    (from_kaggle_sets['Weight'] < 11),
    (from_kaggle_sets['Weight'] < 25),
    (from_kaggle_sets['Weight'] < 45), 
    (from_kaggle_sets['Weight'] >= 45)
]
choices = ['Toy', 'Small', 'Medium', 'Large', 'Giant']

from_kaggle_sets["size_category"] =  np.select(conditions, choices, default='Unknown')
from_kaggle_sets["size_category"].value_counts()

/var/folders/yz/btvl331d4g54624wdzgx_2hc0000gn/T/ipykernel_79685/3934749855.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from_kaggle_sets["size_category"] =  np.select(conditions, choices, default='Unknown')


size_category
Large     416
Medium    222
Giant     201
Small      52
Toy        44
Name: count, dtype: int64

In [129]:
display(from_kaggle_sets.head(10))


,Breed,Age,Weight,Gender,Color,Coat Length,Coat Type,Energy Level,Playfulness Level,size_category
0,Bulldog,10.5,12.7,Female,Black and Tan,Short,Smooth,3.0,4.0,Medium
1,Akita,1.9,39.9,Female,Black and White,Medium,Double,4.0,3.0,Large
2,Pomeranian,12.4,45.4,Female,Tan,Long,Double,3.0,3.0,Giant
3,Siberian Husky,6.3,24.5,Male,Black,Medium,Double,5.0,5.0,Medium
4,Beagle,7.0,53.6,Male,Red,Short,Smooth,4.0,4.0,Giant
6,Alaskan Malamute,4.8,6.4,Male,Tan,Medium,Double,4.0,3.0,Small
7,Shetland Sheepdog,3.3,27.1,Female,Gray,Long,Double,4.0,5.0,Large
8,Shetland Sheepdog,3.3,27.1,Female,Gray,Medium,Double,4.0,3.0,Large
9,Miniature Schnauzer,9.2,23.3,Male,Tricolor,Medium,Wiry,3.0,4.0,Medium
10,Chesapeake Bay Retriever,8.5,42.0,Female,Cream,Medium,Wiry,4.0,3.0,Large


#### ***TODO*** Get Body Condition Score:
- Use mean weight for breed using dog data above - can even put in quantiles

#### ***TODO*** Neutered/Spayed:

In [130]:
N_nonnull = len(notnull)

In [131]:
from scipy.stats import bernoulli
p = 0.75
spayed_samples = bernoulli.rvs(p, size=N_nonnull)
from_kaggle_sets["Spayed/Neutered"] = spayed_samples

/var/folders/yz/btvl331d4g54624wdzgx_2hc0000gn/T/ipykernel_79685/2018436567.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from_kaggle_sets["Spayed/Neutered"] = spayed_samples


In [132]:
from_kaggle_sets["Spayed/Neutered"].value_counts()

Spayed/Neutered
1    718
0    217
Name: count, dtype: int64

In [133]:
indoor_outdoor = np.random.choice(["Indoor_Now", "Outdoor_Now"], size=N_nonnull, p=[0.8,0.2])
season = np.random.choice(["Spring", "Summer", "Fall", "Winter"], size=N_nonnull, p=[0.25,0.25,0.25,0.25])
# can reweight samples to pull from mostly warm months eventually
from_kaggle_sets["Indoor/Outdoor"] = indoor_outdoor
from_kaggle_sets["Season"] = season

/var/folders/yz/btvl331d4g54624wdzgx_2hc0000gn/T/ipykernel_79685/990742337.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from_kaggle_sets["Indoor/Outdoor"] = indoor_outdoor
/var/folders/yz/btvl331d4g54624wdzgx_2hc0000gn/T/ipykernel_79685/990742337.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from_kaggle_sets["Season"] = season


#### Sun Exposure & Wind Cooling

In [134]:

# Generate multiple samples from the normal distribution
import numpy as np

sun_exposure_samples = []
wind_cool_samples = []
this_sun_exposure = 0
this_wind_cool = 0
# Create a default random number generator (recommended for new code)
rng = np.random.default_rng()
for index, row in from_kaggle_sets.iterrows():
    this_season = row["Season"]  # Access the Season column value
    this_indoor_outdoor = row["Indoor/Outdoor"]  # Access the Indoor_Outdoor column value
    if this_indoor_outdoor == "Indoor_Now":  # if dog is indoors
        this_sun_exposure = 0
        this_wind_cool = 0
    else:  # if dog is outdoors
        if this_season == "Winter":
            sun_mu = 1
            sun_sd = 0.3
            wind_mu = -0.8
            wind_sd = 0.4
        elif this_season == "Spring":
            sun_mu = 1.3
            sun_sd = 0.4
            wind_mu = -1.2
            wind_sd = 0.5
        elif this_season == "Summer":
            sun_mu = 1.8
            sun_sd = 0.5
            wind_mu = -1
            wind_sd = 0.5
        elif this_season == "Fall":
            sun_mu = 1.2
            sun_sd = 0.4
            wind_mu = -1
            wind_sd = 0.5
        this_sun_exposure = np.random.normal(sun_mu, sun_sd)
        this_wind_cool = np.random.normal(wind_mu, wind_sd)
    sun_exposure_samples.append(this_sun_exposure)
    wind_cool_samples.append(this_wind_cool)

    
print(len(sun_exposure_samples))
#print(sun_exposure_samples)

print(len(wind_cool_samples))
#print(wind_cool_samples)

from_kaggle_sets["Sun Exposure"] = sun_exposure_samples
from_kaggle_sets["Wind Cooling Effect"] = wind_cool_samples

935
935


/var/folders/yz/btvl331d4g54624wdzgx_2hc0000gn/T/ipykernel_79685/2279897094.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from_kaggle_sets["Sun Exposure"] = sun_exposure_samples
/var/folders/yz/btvl331d4g54624wdzgx_2hc0000gn/T/ipykernel_79685/2279897094.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from_kaggle_sets["Wind Cooling Effect"] = wind_cool_samples


In [135]:
display(from_kaggle_sets[:10])

,Breed,Age,Weight,Gender,Color,Coat Length,Coat Type,Energy Level,Playfulness Level,size_category,Spayed/Neutered,Indoor/Outdoor,Season,Sun Exposure,Wind Cooling Effect
0,Bulldog,10.5,12.7,Female,Black and Tan,Short,Smooth,3.0,4.0,Medium,1,Indoor_Now,Fall,0.000000,0.000000
1,Akita,1.9,39.9,Female,Black and White,Medium,Double,4.0,3.0,Large,1,Outdoor_Now,Spring,1.834451,-0.715264
2,Pomeranian,12.4,45.4,Female,Tan,Long,Double,3.0,3.0,Giant,1,Outdoor_Now,Winter,1.055014,-0.952858
3,Siberian Husky,6.3,24.5,Male,Black,Medium,Double,5.0,5.0,Medium,0,Indoor_Now,Fall,0.000000,0.000000
4,Beagle,7.0,53.6,Male,Red,Short,Smooth,4.0,4.0,Giant,1,Indoor_Now,Winter,0.000000,0.000000
6,Alaskan Malamute,4.8,6.4,Male,Tan,Medium,Double,4.0,3.0,Small,1,Indoor_Now,Spring,0.000000,0.000000
7,Shetland Sheepdog,3.3,27.1,Female,Gray,Long,Double,4.0,5.0,Large,0,Indoor_Now,Spring,0.000000,0.000000
8,Shetland Sheepdog,3.3,27.1,Female,Gray,Medium,Double,4.0,3.0,Large,0,Indoor_Now,Winter,0.000000,0.000000
9,Miniature Schnauzer,9.2,23.3,Male,Tricolor,Medium,Wiry,3.0,4.0,Medium,0,Outdoor_Now,Summer,1.956702,-1.021082
10,Chesapeake Bay Retriever,8.5,42.0,Female,Cream,Medium,Wiry,4.0,3.0,Large,1,Indoor_Now,Spring,0.000000,0.000000


### ***TODO*** Model Activity level using playfulness & energy level (for breed - already in row), and age for simulated dog (older = less active) 

### ***TODO*** Ambient Temp and Humidity (Southern US) -simulating what is measured by device

### ***TODO*** Coat Insulation/ Heat trap feature
- how much warmer/more humid the space near the collar is vs true ambient
- Use coat length (map)
- Thicker/longer coat traps more heat
- Thin coat allows for more air flow
- also do one for humidity

### ***TODO*** Simulate Core Body Temp

In [136]:
# decide how to simulate using breed, coat length, size, and season. 

### Methodology (after initial merges/data engineering from Kaggle):

### 1. Input Features
- **Weather_Temp_station**, **Weather_Hum_station** (from geolocated weather data)
- **Season** (Winter, Spring, Summer, Fall)
- **Indoor flag** (whether the dog primarily occupies indoor or outdoor environments)
- **Coat_Length**, **Coat_Type** (mapped from breed)
- **Energy_Level** (1–5)
- **Playfulness_Level** (1–5)
- **In_Sun flag** (indicator for direct solar exposure)
- *(No wet-fur flag is modeled in this version.)*

---

### 2. Compute `Ambient_Temp_True`
Represents the **true air temperature** around the dog (from weather data + indoor adjustment).

- **If outdoor:**
  \[
  Ambient\_Temp\_True \sim \mathcal{N}(Weather\_Temp\_station, \sigma_{season})
  \]
  Seasonal SDs:
  - Winter = 4 °C  
  - Spring = 4 °C  
  - Summer = 3 °C  
  - Fall = 4 °C

- **If indoor:**
  \[
  Ambient\_Temp\_True \sim \mathcal{N}(22 + 0.2 \times (Weather\_Temp\_station - 22), 2)
  \]
  Uses a 22 °C baseline with weak coupling to outdoor temperature.

---

### 3. Compute `Ambient_Humidity_True`
Represents the **true environmental humidity** (weather-station humidity adjusted for indoor/outdoor).

- **If outdoor:**
  \[
  Ambient\_Humidity\_True \sim \mathcal{N}(Weather\_Hum\_station, 10)
  \]

- **If indoor:**
  \[
  Ambient\_Humidity\_True \sim \mathcal{N}(45 + 0.2 \times (Weather\_Hum\_station - 45), 10)
  \]
  Uses 45 % RH indoor baseline.

---

### 4. Coat-Based Microclimate (“Heat-Trap”) Offsets
Each dog’s coat forms a **microclimate** that traps heat and moisture near the collar.

| Coat Length | Temp Offset (°C) | Humidity Offset (%) |
|--------------|------------------|---------------------|
| Hairless     | +0.2             | +0                  |
| Short        | +0.5             | +2                  |
| Medium       | +1.0             | +4                  |
| Long         | +1.5             | +6                  |
| Arctic       | +2.0             | +8                  |

Small random noise and interactions with extreme ambient heat/humidity can be added.

---

### 5. Collar Device Readings
A sensor **hanging from the dog’s collar** measures the microenvironment **within or just under the fur layer**, not open air.

\[
T_{device} = Ambient\_Temp\_True + temp\_trap + energy\_adj + sun\_adj + \epsilon
\]
\[
H_{device} = Ambient\_Humidity\_True + hum\_trap + energy\_adj\_h + sun\_adj\_h + \epsilon_h
\]

#### Adjustments:
- **Energy / Playfulness factor:**
  \[
  energy\_adj = 0.1 \times (Energy + Playfulness - 6)
  \]
  → higher activity (combined score > 6) slightly increases heat/humidity.

- **Sun exposure:**
  \[
  sun\_adj = 1.5 \text{ °C if In\_Sun = True, else 0}
  \]
  \[
  sun\_adj\_h = 3\% \text{ RH if In\_Sun = True, else 0}
  \]

---

### 6. Simulated Core Body Temperature (CBT)
Estimate CBT as a function of collar temperature and coat insulation.

| Coat Length | Δ (Device → Core) °C |
|--------------|----------------------|
| Hairless     | 0.3 |
| Short        | 0.6 |
| Medium       | 1.0 |
| Long         | 1.6 |
| Arctic       | 2.0 |

\[
CBT = T_{device} + \Delta_{dev\_to\_core} + 0.05 \times (Energy + Playfulness - 6) + \epsilon
\]

Clip CBT to the physiologic range:
\[
37.0 \le CBT \le 40.5
\]

---

### 7. Outputs and Derived Features
Each simulated row will include:

| Variable | Description | Units |
|-----------|--------------|-------|
| `Ambient_Temp_True` | Local air temp (weather + indoor adjustment) | °C |
| `Ambient_Humidity_True` | Local humidity | % |
| `T_device` | Collar sensor temperature | °C |
| `H_device` | Collar sensor humidity | % |
| `CBT` | Core body temperature | °C |
| `Temp_Diff` | T_device − Ambient_Temp_True | °C |
| `Hum_Diff` | H_device − Ambient_Humidity_True | % |
| `Coat_Insulation_Index` | numeric index summarizing coat heat/humidity trap | — |

---

💡 *This version uses energy/playfulness levels instead of direct activity level, treating them as behavioral intensity factors that modestly increase heat and humidity near the sensor.*
